In [2]:
import pandas as pd
import numpy as np

# Importing dataset


In [3]:
# !kaggle datasets download A-d notshrirang/spotify-million-song-dataset

In [4]:
# %unzip spotify-million-song-dataset.zip

In [5]:
spotifyDF = pd.read_csv("/kaggle/input/fulldf/spotify_millsongdata.csv")

# Milestone 3

In [6]:
!pip install transformers

In [7]:
from transformers import AutoTokenizer, TFT5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
model = TFT5ForConditionalGeneration.from_pretrained("google-t5/t5-small")

2024-05-18 15:55:34.942721: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-18 15:55:34.942828: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-18 15:55:35.089771: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [8]:
spotifyDFMS3=spotifyDF.copy()

In [9]:
def split_sentences(df, col_name):
    df['X'] = df[col_name].apply(lambda x: ' '.join(x.split()[:len(x.split())//2]))
    df['Y'] = df[col_name].apply(lambda x: ' '.join(x.split()[len(x.split())//2:]))
    return df

spotifyDFMS3=split_sentences(spotifyDFMS3,"text")
spotifyDFMS3.head(2)

,artist,song,link,text,X,Y
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA...","Look at her face, it's a wonderful face And it...",And when we go for a walk in the park And she ...
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen...","Take it easy with me, please Touch me gently l...",your eyes Like the feeling of a thousand butte...


In [10]:
print(len(spotifyDFMS3["Y"][0])+len(spotifyDFMS3["X"][0]))
print(len(spotifyDFMS3["text"][0]))

719
781


There is a difference in the values since the split_sentences method also removes the extra unneeded tokens such as white spaces adn \n.

In [11]:
spotifyDFMS3['artist_lyric']=spotifyDFMS3["artist"]+" || "+spotifyDFMS3["X"]

In [12]:
spotifyDFMS3.head(2)

,artist,song,link,text,X,Y,artist_lyric
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA...","Look at her face, it's a wonderful face And it...",And when we go for a walk in the park And she ...,"ABBA || Look at her face, it's a wonderful fac..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen...","Take it easy with me, please Touch me gently l...",your eyes Like the feeling of a thousand butte...,"ABBA || Take it easy with me, please Touch me ..."


In [13]:
def max_words(df, col_name):
    max_word_count = df[col_name].apply(lambda x: len(x.split(" "))).max()
    return max_word_count

max_word_count_X = max_words(spotifyDFMS3, 'X')
max_word_count_Y = max_words(spotifyDFMS3, 'Y')
max_word_count = max_words(spotifyDFMS3, 'artist_lyric')
print("X max :", max_word_count_X)
print("Y max :", max_word_count_Y)
print("X AND artist max:",max_word_count)

X max : 413
Y max : 414
X AND artist max: 416


In [14]:
from datasets import load_dataset
spotifyDataset = load_dataset('csv',data_files="/kaggle/input/fulldf/spotify_millsongdata.csv", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [15]:
type(spotifyDataset)

datasets.arrow_dataset.Dataset

In [16]:
def split_sentences(df,OldDF):
    x=[]
    y=[]
    artist_lyric=[]
    for string in OldDF["X"]:
        x.append(string)
    for string in OldDF["Y"]:
        y.append(string)
    for string in OldDF["artist_lyric"]:
        artist_lyric.append(string)
    df = df.add_column("X", x)
    df = df.add_column("Y", y)
    df = df.add_column("artist_lyric", artist_lyric)
    return df

spotifyDataset=split_sentences(spotifyDataset,spotifyDFMS3)

In [17]:
spotifyDataset[0]

{'artist': 'ABBA',
 'song': "Ahe's My Kind Of Girl",
 'link': '/a/abba/ahes+my+kind+of+girl_20598417.html',
 'text': "Look at her face, it's a wonderful face  \r\nAnd it means something special to me  \r\nLook at the way that she smiles when she sees me  \r\nHow lucky can one fellow be?  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?  \r\n  \r\nAnd when we go for a walk in the park  \r\nAnd she holds me and squeezes my hand  \r\nWe'll go on walking for hours and talking  \r\nAbout all the things that we plan  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?\r\n\r\n",
 'X': "Look at her face, it's a wonderful face And it mean

In [18]:
columns_to_drop = ["song", "link","text"]
spotifyDataset = spotifyDataset.remove_columns(columns_to_drop)

In [19]:
spotifyDataset[0]

{'artist': 'ABBA',
 'X': "Look at her face, it's a wonderful face And it means something special to me Look at the way that she smiles when she sees me How lucky can one fellow be? She's just my kind of girl, she makes me feel fine Who could ever believe that she could be mine? She's just my kind of girl, without her I'm blue And if she ever leaves me what could I do, what could I do?",
 'Y': "And when we go for a walk in the park And she holds me and squeezes my hand We'll go on walking for hours and talking About all the things that we plan She's just my kind of girl, she makes me feel fine Who could ever believe that she could be mine? She's just my kind of girl, without her I'm blue And if she ever leaves me what could I do, what could I do?",
 'artist_lyric': "ABBA || Look at her face, it's a wonderful face And it means something special to me Look at the way that she smiles when she sees me How lucky can one fellow be? She's just my kind of girl, she makes me feel fine Who could 

In [20]:
spotifyDataset

Dataset({
    features: ['artist', 'X', 'Y', 'artist_lyric'],
    num_rows: 57650
})

In [21]:
def process_dataset(example):
    artist = example['artist']
    first_half = example['X']
    second_half = example['Y']
    
    input_text = f"{artist}: {first_half}"
    target_text = second_half
    
    return {'input_text': input_text, 'target_text': target_text}

spotifyDataset = spotifyDataset.map(process_dataset, remove_columns=["X", "Y", "artist_lyric"])

Map:   0%|          | 0/57650 [00:00<?, ? examples/s]

In [22]:
spotifyDataset

Dataset({
    features: ['artist', 'input_text', 'target_text'],
    num_rows: 57650
})

In [23]:
def tokenize_function(examples):
    inputs = [f"<s> {text} </s>" for text in examples['input_text']]
    targets = [f"<s> {text} </s>" for text in examples['target_text']]
    model_inputs = tokenizer(inputs, max_length=max_word_count, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=max_word_count, truncation=True, padding="max_length")
    
    model_inputs["labels"] = labels["input_ids"]
    decoder_input_ids = labels['input_ids'].copy()
    decoder_start_token_id = tokenizer.pad_token_id
    
    for idx in range(len(decoder_input_ids)):
        decoder_input_ids[idx] = [decoder_start_token_id] + decoder_input_ids[idx][:-1]
    
    model_inputs["labels"] = labels["input_ids"]
    model_inputs["decoder_input_ids"] = decoder_input_ids

    return model_inputs

spotifyDataset = spotifyDataset.map(tokenize_function, batched=True, remove_columns=["artist", "input_text", "target_text"])

Map:   0%|          | 0/57650 [00:00<?, ? examples/s]

In [24]:
spotifyDataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'],
    num_rows: 57650
})

In [25]:
type(spotifyDataset)

datasets.arrow_dataset.Dataset

In [26]:
spotifyDataset=spotifyDataset.train_test_split(test_size=0.2)

In [27]:
spotifyDataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'],
        num_rows: 46120
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'],
        num_rows: 11530
    })
})

In [28]:
import tensorflow as tf

In [29]:
train_dataset = spotifyDataset["train"].with_format("numpy")[:] 
test_dataset = spotifyDataset["test"].with_format("numpy")[:]

In [30]:
import tensorflow as tf
from tensorflow import keras

optimizer = keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]

In [31]:
!pip install GPUtil

from GPUtil import showUtilization as gpu_usage
gpu_usage()  

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7393 sha256=f4d09b1c391293ef2ac98afa16a9d615449cf92358480682a7e1a56e1348ce4d
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built GPUtil
| ID | GPU | MEM |
------------------
|  0 |  0% | 96% |


In [32]:
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [33]:
model.fit(train_dataset, validation_data=test_dataset,batch_size=16, epochs=3)

Epoch 1/3
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


I0000 00:00:1716047933.435422      89 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2883/2883 [==============================] - 2310s 764ms/step - loss: 1.1300 - sparse_categorical_accuracy: 0.8108 - val_loss: 0.9427 - val_sparse_categorical_accuracy: 0.8343
Epoch 2/3
2883/2883 [==============================] - 2183s 757ms/step - loss: 0.9780 - sparse_categorical_accuracy: 0.8289 - val_loss: 0.9069 - val_sparse_categorical_accuracy: 0.8387
Epoch 3/3
2883/2883 [==============================] - 2186s 758ms/step - loss: 0.9477 - sparse_categorical_accuracy: 0.8329 - val_loss: 0.8882 - val_sparse_categorical_accuracy: 0.8412


In [55]:
inputs = tokenizer(" Donna Summer || The club isn't the best place to find ",return_tensors="tf").input_ids
outputs = model.generate(inputs, max_new_tokens=50, do_sample=True, top_k=30, top_p=0.95)
tokenizer.decode(outputs[0], skip_special_tokens=True)

"Donna Summer || Donna Summer || You don't want it all Donna Summer || She's the club a lot you adore Donna Summer || Donna Summer || Donna Summer || Donna Summer || Donna Summer ||"

> ***6:56 Fit***

In [58]:
import tensorflow as tf

def evaluate_lyric_generation(model, tokenizer, lyrics_df, num_samples=10):
  prompts = lyrics_df.sample(num_samples)["artist_lyric"].tolist()  # Replace with your actual column name

  # Generate lyrics for each prompt
  generated_lyrics = []
  for prompt in prompts:
    # Preprocess the prompt
    input_ids = tokenizer(prompt, return_tensors="tf").input_ids

    # Generate lyrics using the model
    generated_sequence = model.generate(
        input_ids,
        max_length=100,  # Adjust this based on your desired lyric length
        do_sample=True,
        top_k=50,
        top_p=0.9,
        num_return_sequences=1
    )

    # Decode the generated sequence
    decoded_lyric = tokenizer.decode(generated_sequence[0], skip_special_tokens=True)

    # Add prompt and generated lyric to dictionary
    generated_lyrics.append({
        "prompt": prompt,
        "generated_lyric": decoded_lyric
    })

  return generated_lyrics

generated_lyrics = evaluate_lyric_generation(model, tokenizer, spotifyDFMS3)

# Print the generated lyrics for evaluation
for lyric in generated_lyrics:
  print(f"Prompt: {lyric['prompt']}")
  print(f"Generated Lyric: {lyric['generated_lyric']}")
  print("-" * 20)

Prompt: Owl City || There was a shot in the dark I was caught by surprise There was a hole in my heart There were tears in your eyes And there was nothing to say 'Cause you made up your mind And so I guess what you meant When you left was goodbye So if you're gonna go And leave me in a lonely gray I won't let it show Until you finally float away You gotta know that you tear me up When you say you wanna take it all away, take it all away You tear it up when you say You wanna take it all away, take it all away! You were the one, and it was enough To be the one you were dreaming of! You were the one and we called it love And now you take it away, take away! I felt a pain in my chest with your kiss on my cheek And as I tried to digest the words I couldn't believe I'm left with nothing to say with my heart on my sleeve Making it easy to hurt, and even harder to breathe
Generated Lyric: You gotta know that you tear me up When you say you wanna take it all away, take it all away You tear it u